Imports

In [1]:
import pandas as pd

Now, as we scraped data from 4 different webpages, we try to match same matches as each of the webpages names them slightly differently. 
First, we find the betting office with the highest amount of available matches.
Then, we find unique dates and playtimes.
Next, we try to match the matches by playtime across our 4 different datasets. 
Now, we try to ensure we matched the identical matches by looking for matches starting with the same 1st word among the ones with the same playtime.
If we get 1 exact match, we are happy. 
We save these matches into csv_final

In [2]:
def analyze_exchange_profit_for_sport(sport):
    #empty array that will be used for sport_final dataset
    rowsToAppend = []
    rowsToAppend.append(["Match_name", "Date", "Chance", "Ifortuna", "Synot", "Tipsport"])
    chance = pd.read_csv('chance/' + sport + '.csv')
    ifortuna = pd.read_csv('ifortuna/' + sport + '.csv')
    synottip = pd.read_csv('synottip/' + sport + '.csv')
    tipsport = pd.read_csv('tipsport/' + sport + '.csv')
    chance = chance.sort_values(by=['date'])
    ifortuna = ifortuna.sort_values(by=['date'])
    synottip = synottip.sort_values(by=['date'])
    tipsport = tipsport.sort_values(by=['date'])
#setting max to -1 as we want to be sure this value will be overwritten by at least 0 matches at a specific playtime
#we need to set "largest" value even if there are no matches for the code to work
max = -1
    if (chance.size > max):
        largest = chance
        max = chance.size
    if (ifortuna.size > max):
        largest = ifortuna
        max = ifortuna.size
    if (synottip.size > max):
        largest = synottip
        max = synottip.size
    if (tipsport.size > max):
        largest = tipsport
    dates = largest['date'].unique()
    #create new dataframe for each playtime for each webpage
    for date in dates:
        matches_for_date_chance = chance.loc[chance['date'] == date]
        matches_for_date_ifortuna = ifortuna.loc[ifortuna['date'] == date]
        matches_for_date_synottip = synottip.loc[synottip['date'] == date]
        matches_for_date_tipsport = tipsport.loc[tipsport['date'] == date]
        #find the webpage with highest no. of observations for a specific playtime
        max = -1
        if (matches_for_date_chance.size > max):
            largest = matches_for_date_chance
            max = matches_for_date_chance.size
        if (matches_for_date_ifortuna.size > max):
            largest = matches_for_date_ifortuna
            max = matches_for_date_ifortuna.size
        if (matches_for_date_synottip.size > max):
            largest = matches_for_date_synottip
            max = matches_for_date_synottip.size
        if (matches_for_date_tipsport.size > max):
            largest = matches_for_date_tipsport
        names = largest['name'].unique()
        #second forloop to find the exact match by 1st word in the matches' name
        for name in names:
            try:
                #create blank array and add name and date
                row = []
                row.append(name)
                row.append(date)
                # here we try to clean the names of the matches of special symbols, that could complicate the matching process
                # parsed = divide name into single words and pick the 1st one 
                parsed = str(name).replace(',', ' ').replace('/', ' ').split(' ')
                identifier = parsed[0]
                #for each url we try to find this match with this specific identifier
                matches_for_date_chance_select = matches_for_date_chance.loc[
                    matches_for_date_chance['name'].str.contains(identifier)]
                #when we find exactly one match, we are happy, and save it as a list
                if (len(matches_for_date_chance_select.index) == 1):
                    values = matches_for_date_chance_select.iloc[0].values.tolist()
                    #crucial formula = this formula evaluates the exchange profit/margin of individual booking companies 
                    #as we mentioned before, there still may be observations of bets, that do not bet on win-lose
                    #such are bets on final winner etc. therefore, we rule out these bets with following condition
                    #as these bets will have rates that fail to comply with our ratio (will be below 1)
                    #we save these ratios for each match for each company if available, otherwise append zero
                    exchangeProfit = 1/float(values[2]) + 1/float(values[4])
                    if (exchangeProfit > 1):
                        row.append(exchangeProfit)
                    else:
                        row.append(0)
                else:
                    row.append(0)
                #repeat for all the betting offices
                matches_for_date_ifortuna_select = matches_for_date_ifortuna.loc[
                    matches_for_date_ifortuna['name'].str.contains(identifier)]
                if (len(matches_for_date_ifortuna_select.index) == 1):
                    values = matches_for_date_ifortuna_select.iloc[0].values.tolist()
                    exchangeProfit = 1 / float(values[2]) + 1 / float(values[4])
                    if (exchangeProfit > 1):
                        row.append(exchangeProfit)
                    else:
                        row.append(0)
                else:
                    row.append(0)

                matches_for_date_synottip_select = matches_for_date_synottip.loc[
                    matches_for_date_synottip['name'].str.contains(identifier)]
                if (len(matches_for_date_synottip_select.index) == 1):
                    values = matches_for_date_synottip_select.iloc[0].values.tolist()
                    exchangeProfit = 1 / float(values[2]) + 1 / float(values[4])
                    if (exchangeProfit > 1):
                        row.append(exchangeProfit)
                    else:
                        row.append(0)
                else:
                    row.append(0)

                matches_for_date_tipsport_select = matches_for_date_tipsport.loc[
                    matches_for_date_tipsport['name'].str.contains(identifier)]
                if (len(matches_for_date_tipsport_select.index) == 1):
                    values = matches_for_date_tipsport_select.iloc[0].values.tolist()
                    exchangeProfit = 1 / float(values[2]) + 1 / float(values[4])
                    if (exchangeProfit > 1):
                        row.append(exchangeProfit)
                    else:
                        row.append(0)
                else:
                    row.append(0)
                
                #observations with incorrect/zero values shall not be appended
                if(row[2] == 0 and row[3] == 0 and row[4] == 0 and row[5] == 0):
                    continue
                else:
                    rowsToAppend.append(row)
            except:
                print()
    df = pd.DataFrame(rowsToAppend)
    df.to_csv(sport + '_final.csv', index=False, header=False)

Run individual analyses

In [3]:
analyze_exchange_profit_for_sport('tenis')
analyze_exchange_profit_for_sport('voleyball')